In [14]:
import yt_dlp as youtube_dl
import pandas as pd
import numpy as np
import os
import librosa

import pickle

from tqdm import tqdm
from IPython.display import clear_output

## Загружаем список всех меток для аудио

In [28]:
labels_dict = pd.read_csv('audioset/class_labels_indices.csv', index_col=0)
display(labels_dict)

,mid,display_name
index,,
0,/m/09x0r,Speech
1,/m/05zppz,"Male speech, man speaking"
2,/m/02zsn,"Female speech, woman speaking"
3,/m/0ytgt,"Child speech, kid speaking"
4,/m/01h8n0,Conversation
...,...,...
522,/m/07p_0gm,Throbbing
523,/m/01jwx6,Vibration
524,/m/07c52,Television


## Фильтруем метки

* Занимался этим вручную. Убрал все музыкальные инструменты, звуки которые мне не знакомы или на мой взгляд неоднозначны и метки значение которых вообще не понял. И по возможности группировал остальные.
* Получилось 65 групп.

In [35]:
def print_labels_groups(labels_groups):
    for i, group in enumerate(labels_groups):
        print(i, group['group_name'], group['labels'])

def proc_label(label_name, labels_groups):
    operation = input('label \'{}\'  save (s) or delete (d) or add to group(a):'.format(label_name))
    
    if operation == 's':
        group_name = input('group name:')
        row['group_name'] = group_name
        row['labels'].append(label_name)
        labels_groups.append(row)
        
    elif operation == 'd':
        print()
        
    elif operation == 'a':
        group = int(input('group id:'))
        labels_groups[group]['labels'].append(label_name)
        
    else:
        print('invalid value passed')
        return False
        
    clear_output(wait=True)
    return True

def try_proc_label(label_name, labels_groups):
    try:
        return proc_label(label_name, labels_groups)
    except ValueError:
        return False
    
labels_groups = []
for i in labels_dict.index:
    row = {'group_name' : '', 'labels' : []}
    
    print_labels_groups(labels_groups)
    print()
    
    Processed = False
    while not Processed:
        Processed = try_proc_label(labels_dict['display_name'][i], labels_groups)
        
print_labels_groups(labels_groups)

0 Speech ['Speech', 'Male speech, man speaking', 'Female speech, woman speaking', 'Child speech, kid speaking', 'Conversation', 'Narration, monologue', 'Babbling', 'Speech synthesizer', 'Patter']
1 Shout ['Shout', 'Battle cry', 'Children shouting', 'Screaming']
2 Whispering ['Whispering']
3 Laughter ['Laughter', 'Baby laughter', 'Giggle', 'Belly laugh', 'Chuckle, chortle']
4 Crying ['Crying, sobbing', 'Baby cry, infant cry', 'Whimper']
5 Moan ['Wail, moan', 'Groan']
6 Sigh ['Sigh']
7 Singing ['Singing', 'Choir', 'Chant', 'Male singing', 'Female singing', 'Child singing', 'Synthetic singing', 'Humming']
8 Grunt ['Grunt']
9 Whistling ['Whistling']
10 Snoring ['Snoring']
11 Gasp ['Gasp']
12 Cough ['Cough', 'Throat clearing']
13 Sneeze ['Sneeze']
14 footsteps ['Walk, footsteps']
15 Gargling ['Gargling']
16 Stomach rumble ['Stomach rumble']
17 Burping, eructation ['Burping, eructation']
18 Hiccup ['Hiccup']
19 Finger snapping ['Finger snapping']
20 Clapping ['Clapping', 'Cheering', 'Applaus

In [39]:
pd.DataFrame(labels_groups).to_csv('audioset/Processed/labels_groups.csv')

## В группах сопоставояем название метки и ее id

In [48]:
def print_ids_groups(ids_groups):
    for i, group in enumerate(ids_groups):
        print(i, group['group_name'], group['ids'])
        
def find_ids(labels_df, label_name):
    label = labels_df.loc[labels_df['display_name'] == label_name]
    return label['mid'].values[0]

ids_groups = []
for group in labels_groups:
    ids = [find_ids(labels_dict, label_name) for label_name in group['labels']]
    
    row = {'group_name' : group['group_name'], 'ids' : ids}
    ids_groups.append(row)
        
print_ids_groups(ids_groups)
pd.DataFrame(ids_groups).to_csv('audioset/Processed/ids_groups.csv')

0 Speech ['/m/09x0r', '/m/05zppz', '/m/02zsn', '/m/0ytgt', '/m/01h8n0', '/m/02qldy', '/m/0261r1', '/m/0brhx', '/m/07r4gkf']
1 Shout ['/m/07p6fty', '/m/04gy_2', '/t/dd00135', '/m/03qc9zr']
2 Whispering ['/m/02rtxlg']
3 Laughter ['/m/01j3sz', '/t/dd00001', '/m/07r660_', '/m/07sq110', '/m/07rgt08']
4 Crying ['/m/0463cq4', '/t/dd00002', '/m/07qz6j3']
5 Moan ['/m/07qw_06', '/m/07s2xch']
6 Sigh ['/m/07plz5l']
7 Singing ['/m/015lz1', '/m/0l14jd', '/m/02bk07', '/t/dd00003', '/t/dd00004', '/t/dd00005', '/t/dd00006', '/m/02fxyj']
8 Grunt ['/m/07r4k75']
9 Whistling ['/m/01w250']
10 Snoring ['/m/01d3sd']
11 Gasp ['/m/07s0dtb']
12 Cough ['/m/01b_21', '/m/0dl9sf8']
13 Sneeze ['/m/01hsr_']
14 footsteps ['/m/07pbtc8']
15 Gargling ['/m/0939n_']
16 Stomach rumble ['/m/01g90h']
17 Burping, eructation ['/m/03q5_w']
18 Hiccup ['/m/02p3nc']
19 Finger snapping ['/m/025_jnm']
20 Clapping ['/m/0l15bq', '/m/053hz1', '/m/028ght']
21 heartbeat ['/m/01jg02', '/m/01jg1z']
22 Bark ['/m/05tny_', '/m/07rc7d9']
23 Howl

## Загружаю данные об аудио, метки которых есть среди оставленных на прошлом шаге.

In [57]:
def proc_id(audio_info):
    audio_info['id'] = [origin_id[:origin_id.rfind('_')] for origin_id in audio_info['segment_id'].values]
    return audio_info

def find_ids_name(ids_groups, label_id):
    for group in ids_groups:
        if label_id in group['ids']:
            return group['group_name']
        
    return 'unknown id'

def read_audio_info(ids_groups, cnt = None):
    audio_info = pd.read_csv('audioset/audioset_train_strong.tsv', nrows=cnt, sep='\t')
    
    proc_id(audio_info)
    
    using_ids_list = sum(pd.DataFrame(ids_groups)['ids'].values, [])
    
    audios = [i for i in audio_info.index if audio_info['label'][i] in using_ids_list]
    
    audio_info = audio_info.loc[audios]
    
    audio_info['labels_name'] = [find_ids_name(ids_groups, label) for label in audio_info['label']]
    
    return audio_info

audio_info = read_audio_info(ids_groups)
display(audio_info)

,segment_id,start_time_seconds,end_time_seconds,label,id,labels_name
1,b0RFKhbpFJA_30000,4.753,5.720,/m/05zppz,b0RFKhbpFJA,Speech
2,b0RFKhbpFJA_30000,0.000,10.000,/m/07pjwq1,b0RFKhbpFJA,"Fly, housefly"
3,b0RFKhbpFJA_30000,6.899,7.010,/m/07qjznt,b0RFKhbpFJA,Clock
12,NQNTnl0zaqU_70000,4.944,6.951,/m/0284vy3,NQNTnl0zaqU,Toot
17,4PPmyY_-YrA_30000,7.983,8.161,/m/07qjznt,4PPmyY_-YrA,Clock
...,...,...,...,...,...,...
934809,cq-vfngNXMc_70000,3.937,4.059,/m/07qjznt,cq-vfngNXMc,Clock
934811,cq-vfngNXMc_70000,4.652,5.432,/m/05zppz,cq-vfngNXMc,Speech
934813,cq-vfngNXMc_70000,5.399,6.179,/m/01j3sz,cq-vfngNXMc,Laughter
934816,cq-vfngNXMc_70000,7.836,8.015,/m/07qjznt,cq-vfngNXMc,Clock


## Скачиваю аудио, пока не получу 10 Гб данных.

In [74]:
def download_audio(audio, dst_dir):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        }],
        'outtmpl': os.path.join(dst_dir, '%(id)s.%(ext)s'),
    }
    ydl = youtube_dl.YoutubeDL(ydl_opts)
    
    try:
        url = 'http://www.youtube.com/watch?v=' + audio
        info = ydl.extract_info(url, download=False)
        ydl.download([url])
        return info
    except youtube_dl.utils.DownloadError: 
        print('audio', audio, 'loading failed')
        return None
    
def download_audios(audio_info):
    dst_dir = 'audioset/Audio/'
    
    total_size = 0
    
    Gb_modify = 1024 * 1024 * 1024
    
    audio_names = set(audio_info['segment_id'].values)
    for audio_name in audio_names:
        print('Downloaded size: ', total_size / Gb_modify, 'Gb')
        
        info = download_audio(audio_name, dst_dir)
        if info is None:
            clear_output(wait=True)
            continue
        
        dst_file = dst_dir + info['id'] + '.mp3'
        if os.path.exists(dst_file):
            total_size += os.path.getsize(dst_file)
        
        if total_size >= 10 * Gb_modify:
            print('Size limit reached', total_size / Gb_modify, 'Gb')
            print('Stop at', audio_name)
            break
            
        clear_output(wait=True)
            
    Gb_modify = 1024 * 1024 * 1024
    print('Total size: ', total_size / Gb_modify, 'Gb')
    
download_audios(audio_info)

Downloaded size:  9.998074715025723 Gb
[youtube] Extracting URL: http://www.youtube.com/watch?v=ivyIvYnSceU_390000
[youtube] ivyIvYnSceU: Downloading webpage
[youtube] ivyIvYnSceU: Downloading ios player API JSON
[youtube] ivyIvYnSceU: Downloading android player API JSON
[youtube] ivyIvYnSceU: Downloading m3u8 information
[youtube] Extracting URL: http://www.youtube.com/watch?v=ivyIvYnSceU_390000
[youtube] ivyIvYnSceU: Downloading webpage
[youtube] ivyIvYnSceU: Downloading ios player API JSON
[youtube] ivyIvYnSceU: Downloading android player API JSON
[youtube] ivyIvYnSceU: Downloading m3u8 information
[info] ivyIvYnSceU: Downloading 1 format(s): 251
[download] Destination: audioset\Audio\ivyIvYnSceU.webm
[download] 100% of   19.84MiB in 00:00:03 at 5.59MiB/s   
[ExtractAudio] Destination: audioset\Audio\ivyIvYnSceU.mp3
Deleting original file audioset\Audio\ivyIvYnSceU.webm (pass -k to keep)
Size limit reached 10.011680888012052 Gb
Stop at ivyIvYnSceU_390000
Total size:  10.011680888012

## Получаю мелспектрограмму для каждого скачаного звука.

* Загружаю аудио.
* Нахожу его имя в списке с данными, получеными выше и вытаскиваю помеченые части.
* Вычисляю мелспектрограмму, с размером кадра примерно в 100 мсек (кол-во отсчетов равное степени 2) и количеством мелов равным 128.
* В качестве метки ставлю ее порядковый номер в сете из всех имеющихся меток.

In [75]:
def make_frame_size(frame_sec, sr):
    frame = 1
    while frame * 2 <= frame_sec * sr:
        frame *= 2
    return frame

def make_features(wav, sr, cnt = 128, frame_sec=0.1):
    frame_size = make_frame_size(frame_sec, sr)
    mel_spec = librosa.feature.melspectrogram(y=wav, sr=sr, n_mels= cnt, n_fft=frame_size, hop_length=frame_size // 2)
    return np.transpose(mel_spec).tolist()

def make_data_for_audio(y, sr, audio_sectors, labels_dict):
    data = []
    for sector in audio_sectors:
        l = int(round(sector[1] * sr))
        r = int(round(sector[2] * sr))
        
        features = make_features(y[l:r], sr)
        
        for frame in features:
            row = {}
            row['features'] = frame
            row['label'] = labels_dict[sector[3]]
            data.append(row)
    
    return data

def make_data(audio_info):
    res = []
    audio_dir = 'audioset/Audio/'
    labels_dict = {label: i for i, label in enumerate(set(audio_info['label'].values))}
    
    for file_name in tqdm(os.listdir(audio_dir)):
        y, sr = librosa.load(audio_dir + file_name)
        
        audio_id = file_name[:file_name.rfind('.')]
        audio_sectors = audio_info.loc[audio_info['id'] == audio_id].values
        
        res += make_data_for_audio(y, sr, audio_sectors, labels_dict)
        clear_output(wait=True)
        
    return res

data = make_data(audio_info)

100%|██████████████████████████████████████████████████████████████████████████████| 2438/2438 [21:18<00:00,  1.91it/s]


## Проверяю сколько и каких звуков имеется

In [85]:
labels_cnt = {label:len(audio_info.loc[audio_info['labels_name'] == label]) for i, label in enumerate(set(audio_info['labels_name'].values))}
assert(len(audio_info.index) == sum([labels_cnt[key]  for key in labels_cnt]))
labels_cnt

{'Knock': 432,
 'Croak': 876,
 'Thunder': 421,
 'Purr': 761,
 'Gurgling': 352,
 'Burping, eructation': 855,
 'Quack': 894,
 'Howl': 898,
 'Singing': 21932,
 'Moo': 768,
 'Crying': 3150,
 'Caw': 1855,
 'Growling': 1687,
 'Slap, smack': 673,
 'Clock': 58764,
 'Ding-dong': 129,
 'Gargling': 73,
 'Meow': 776,
 'Bleat': 1708,
 'Shout': 8332,
 'Cough': 2418,
 'Sigh': 202,
 'Neigh, whinny': 521,
 'Boiling': 218,
 'Ocean': 1010,
 'Sneeze': 505,
 'Whistling': 2862,
 'Grunt': 347,
 'Speech': 165177,
 'Snoring': 1158,
 'Whispering': 1645,
 'Beep, bleep': 6682,
 'Gunshot, gunfire': 5274,
 'Rain': 1084,
 'Bark': 6340,
 'Crowing, cock-a-doodle-doo': 636,
 'Pour': 710,
 'Bird vocalization, bird call, bird song': 26844,
 'Laughter': 15534,
 'Oink': 1575,
 'Hiccup': 632,
 'Crackle': 336,
 'Moan': 771,
 'Keys jangling': 475,
 'Squish': 133,
 'Squeak': 926,
 'Hiss': 427,
 'Whoosh, swoosh, swish': 517,
 'Stomach rumble': 317,
 'heartbeat': 4830,
 'Finger snapping': 714,
 'Toot': 1477,
 'Waterfall': 259,
 

## Создаю и сохраняю словарь из имени метки и ее порядкового номера.

In [90]:
actual_dict = {label:i for i, label in enumerate(set(audio_info['labels_name'].values))}

## Сохраняю фичи и актуальный словарь меток.

In [91]:
def pkl_write(file_name, data):
    return pickle.dump(data, open(file_name, 'wb'))

pkl_write('audioset/Processed/features.pkl', data)
pkl_write('audioset/Processed/actual_dict.pkl', actual_dict)

In [92]:
actual_dict = pickle.load(open('audioset/Processed/actual_dict.pkl','rb'))
actual_dict

{'Knock': 0,
 'Croak': 1,
 'Thunder': 2,
 'Purr': 3,
 'Gurgling': 4,
 'Burping, eructation': 5,
 'Quack': 6,
 'Howl': 7,
 'Singing': 8,
 'Moo': 9,
 'Crying': 10,
 'Caw': 11,
 'Growling': 12,
 'Slap, smack': 13,
 'Clock': 14,
 'Ding-dong': 15,
 'Gargling': 16,
 'Meow': 17,
 'Bleat': 18,
 'Shout': 19,
 'Cough': 20,
 'Sigh': 21,
 'Neigh, whinny': 22,
 'Boiling': 23,
 'Ocean': 24,
 'Sneeze': 25,
 'Whistling': 26,
 'Grunt': 27,
 'Speech': 28,
 'Snoring': 29,
 'Whispering': 30,
 'Beep, bleep': 31,
 'Gunshot, gunfire': 32,
 'Rain': 33,
 'Bark': 34,
 'Crowing, cock-a-doodle-doo': 35,
 'Pour': 36,
 'Bird vocalization, bird call, bird song': 37,
 'Laughter': 38,
 'Oink': 39,
 'Hiccup': 40,
 'Crackle': 41,
 'Moan': 42,
 'Keys jangling': 43,
 'Squish': 44,
 'Squeak': 45,
 'Hiss': 46,
 'Whoosh, swoosh, swish': 47,
 'Stomach rumble': 48,
 'heartbeat': 49,
 'Finger snapping': 50,
 'Toot': 51,
 'Waterfall': 52,
 'Gasp': 53,
 'Snake': 54,
 'Fly, housefly': 55,
 'Train wheels squealing': 56,
 'Zipper (c